In [1]:

import pandas as pd
import plotly.express as px
import os, sys
from prophet import Prophet
from datetime import timedelta

# Pour import depuis src/
sys.path.append(os.path.abspath("../src"))
from weather_connector import get_weather_forecast


In [2]:

# 📍 Ville cible
ville = "Tunis"
data = get_weather_forecast(ville)
df = pd.DataFrame(data)
df["datetime"] = pd.to_datetime(df["datetime"])
df = df[["datetime", "temp"]].rename(columns={"datetime": "ds", "temp": "y"})
df.head()


,ds,y
0,2025-08-03 21:00:00,29.73
1,2025-08-04 00:00:00,28.05
2,2025-08-04 03:00:00,25.95
3,2025-08-04 06:00:00,26.27
4,2025-08-04 09:00:00,31.57


In [3]:

# 📈 Création et entraînement du modèle Prophet
model = Prophet()
model.fit(df)


19:15:15 - cmdstanpy - INFO - Chain [1] start processing
19:15:17 - cmdstanpy - INFO - Chain [1] done processing


In [4]:

# 🔮 Prédire les prochaines 72 heures (~24 points de 3h)
future = model.make_future_dataframe(periods=24, freq='3H')
forecast = model.predict(future)
forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail()


c:\projet_stage_ete_2025\.venv\Lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(


,ds,yhat,yhat_lower,yhat_upper
59,2025-08-11 06:00:00,24.466512,23.388247,25.480710
60,2025-08-11 09:00:00,29.663755,28.551420,30.778194
61,2025-08-11 12:00:00,33.653001,32.524568,34.749878
62,2025-08-11 15:00:00,35.412267,34.196942,36.535987
63,2025-08-11 18:00:00,32.059644,30.882385,33.299219


In [5]:

# 📊 Visualisation des prédictions avec Plotly
fig = px.line()
fig.add_scatter(x=forecast["ds"], y=forecast["yhat"], name="Prévision")
fig.add_scatter(x=df["ds"], y=df["y"], name="Temp réelle")
fig.update_layout(title=f"🔮 Prévision de température – {ville}", xaxis_title="Date", yaxis_title="Température (°C)")
fig.show()


In [6]:

# Comparaison sur les points connus
merged = pd.merge(df, forecast[["ds", "yhat"]], on="ds", how="left")
merged.dropna(inplace=True)
mae = abs(merged["y"] - merged["yhat"]).mean()
print(f"📉 MAE (Erreur absolue moyenne) : {mae:.2f} °C")


📉 MAE (Erreur absolue moyenne) : 0.43 °C
